In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [3]:
train_parts = []
test_parts = []
with open('/Users/vita/Downloads/flo/train_subsample.jsonl', 'r') as train_file, open('/Users/vita/Downloads/flo/test_subsample.jsonl', 'r') as test_file:
    for train_part, test_part in zip(tqdm(train_file), test_file):
        train_part = json.loads(train_part)
        test_part = json.loads(test_part)
        train_parts.append(train_part)
        test_parts.append(test_part)


0it [00:00, ?it/s]
96it [00:00, 952.91it/s]
197it [00:00, 973.32it/s]
292it [00:00, 963.03it/s]
384it [00:00, 950.53it/s]
479it [00:00, 949.89it/s]
585it [00:00, 966.81it/s]
688it [00:00, 974.97it/s]
784it [00:00, 972.84it/s]
877it [00:00, 966.26it/s]
974it [00:01, 966.61it/s]
1070it [00:01, 965.24it/s]
1170it [00:01, 968.16it/s]
1273it [00:01, 973.06it/s]
1372it [00:01, 972.66it/s]
1470it [00:01, 969.27it/s]
1567it [00:01, 968.64it/s]
1673it [00:01, 973.38it/s]
1772it [00:01, 966.86it/s]
1868it [00:01, 965.91it/s]
1964it [00:02, 964.93it/s]
2064it [00:02, 965.91it/s]
2161it [00:02, 965.77it/s]
2258it [00:02, 964.12it/s]
2362it [00:02, 967.09it/s]
2460it [00:02, 966.96it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance

In [5]:
def metric(real, predicted):
    assert len(real) == len(predicted)
    
    users_mae_values = []
    for real_labels, predicted_labels in zip(real, predicted):
        assert len(real_labels) == len(predicted_labels)
        real_labels, predicted_labels = np.array(real_labels), np.array(predicted_labels)
        
        user_mae = np.mean(np.abs(real_labels - predicted_labels))
        users_mae_values.append(user_mae)
    
    return np.mean(users_mae_values)

### baseline - среднее всех предыдущих циклов

In [78]:
real = []
predicted = []
for train_part in tqdm(train_parts):
    days = map(lambda x: (x['lengths_history'], x['label']), train_part)
    days = filter(lambda x: len(x[0]) > 0, days)
    days = map(lambda x: (np.mean(x[0]), x[1]), days)
    days = np.array(list(days)).T
    if days.shape[0] > 0:
        real.append(days[1])
        predicted.append(days[0])

100%|██████████| 129593/129593 [03:47<00:00, 570.22it/s]


In [79]:
print("baseline = {}".format(metric(real, predicted)))

baseline = 4.308354266098735


### boosting

### first RNN